In [147]:
from bs4 import BeautifulSoup
import requests
import json
import time
import random
import pandas as pd
from sys import exit

In [ ]:
BASE_URL = 'https://www.amazon.es'

In [141]:
api = requests.session()

In [130]:
dfs = pd.read_csv("proxy-list.csv")
proxys_list = dfs["proxys"].to_list()

In [128]:
df = pd.read_csv("list.csv")
headers_list = df["columns"].to_list()

In [144]:
def requests_web(url):
    headers = {
        'User-Agent': random.choice(headers_list)
    }

    proxy = random.choice(proxys_list)
    requests.proxies = proxy
    web = api.get(url, headers=headers, verify=False)
    print(web.status_code)
    return web

In [ ]:
def soup_file(get_Response):
    soup = BeautifulSoup(get_Response.text, 'lxml')
    return soup

In [ ]:
def get_all_categorias(soup):
    categorias_lista = soup.find_all("div", class_="popover-grouping")
    return categorias_lista

In [ ]:
def excluir_categorias(categorias):
    exclusiones = ["Amazon Prime Video", "Amazon Music", "Amazon Photos", "Appstore para Android", "Amazon Business", "Cheques Regalo y Recargas", "E-readers y eBooks Kindle", "Juguetes y Bebé", "Alimentación y bebidas", "Moda", "Handmade", "Amazon Launchpad"]
    categorias_info = []
    for categoria in categorias:
        categorias_terminado = {}
        titulo = categoria.find("h2")
        validador = any(titulo.text in exclusion for exclusion in exclusiones)
        if validador:
            continue
        else:
            categorias_validas = titulo.parent
            categorias_validas_titulo = categorias_validas.find("h2").text
            links_list = categorias_validas.find_all("a")
            categorias_info.append(categorias_validas)
    return categorias_info

In [ ]:
def categorias_info():
    web = requests_web('https://www.amazon.es/gp/site-directory?ref_=nav_em__fullstore_0_1_1_35')
    soup = soup_file(web)
    all_categorias = get_all_categorias(soup)
    categorias = excluir_categorias(all_categorias)
    sub_categorias = verificar_sub_categoria_1(categorias)
    sub_categorias_2 = verificar_sub_categoria_2(sub_categorias)
    print(sub_categorias_2)
    
    '''
    categorias_terminado['titulo_principal'] = titulo
    categorias_terminado['sub_categoria'] = sub_categoria
    categorias_terminado['sub_categoria']['sub_categoria_2'] = sub_categoria_2
    categorias_infos.append(categorias_terminado)
    
    return categorias_infos
    '''

In [ ]:
def verificar_sub_categoria_1(categorias):
    lista_categorias = []
    for categoria in categorias:
        titulo = categoria.find("h2").text
        print(f"\nTitulo principal: {titulo}")
        links_list = categoria.find_all("a") 
        sub_categoria = get_all_sub_categorias(links_list)
        print("  └─ " + f"Cantidad Subcategorias: {len(sub_categoria)}")
        for titulo in sub_categoria:
            print("  └─ " + titulo['titulo'])
        lista_categorias.append(sub_categoria)
    return lista_categorias

In [114]:
def verificar_sub_categoria_2(categorias):
    time.sleep(6)
    sub_categoria = []
    for categoria in categorias:
        for elemento in categoria:
            sub_categoria_2 = {}
            normal = get_sub_categoria(elemento['link'])
            print(normal)
            if normal != []:
                print('Normal')
                print(normal)
                sub_categoria_2['sub_categoria'] = normal
            else:
                print('Departamento')
                departamento = get_departamento_categorias(elemento['link'])
                print(departamento)
                sub_categoria_2['sub_categoria'] = departamento
            sub_categoria.append(sub_categoria_2)

    #json_data = json.dumps(sub_categoria)
    #json_loads = json.loads(json_data)
    return sub_categoria

In [ ]:
def get_all_sub_categorias(links_list):
    links_info = []
    for link in links_list:
        subcategoria = {}
        subcategoria['titulo'] = link.text.strip()
        subcategoria['link'] = BASE_URL + link['href']
        links_info.append(subcategoria)
    return links_info

In [151]:
def get_sub_categoria(url):
    web = requests_web(url)
    soup = soup_file(web)
    print(url)
    sub_categorias_info = []
    try:
        categorias = soup.find("div", class_="left_nav browseBox")
        categorias_titulos = categorias.find_all("h3")
        sub_categorias = categorias.find_all("a")
        for sub_categoria in sub_categorias:
            json_data = {}
            json_data['titulo'] = sub_categoria.text
            json_data['link'] = BASE_URL + sub_categoria['href']
            sub_categorias_info.append(json_data)
        return sub_categorias_info
    except Exception as e:
    
        with open('file.html', 'w') as file:
            file.write(web.text)
            exit()
            
        print(e)

In [148]:
def get_departamento_categorias(url):
    web = requests_web(url)
    print(url)
    soup = soup_file(web)
    try:
        categoria_base = soup.find_all("span", class_="a-size-base a-color-base a-text-bold")
        print(categoria_base)
        for categorias in categoria_base:
            categorias_titulo = categorias.text
            if categorias_titulo == 'Departamento':
                categoria_padre = categorias.parent.parent
                sub_categorias = categoria_padre.find_all("a")
                sub_categorias_info = []
                for sub_categoria in sub_categorias:
                    json_data = {}
                    json_data['titulo'] = sub_categoria.text.strip()
                    json_data['link'] = BASE_URL + sub_categoria['href']
                    sub_categorias_info.append(json_data)
                print(sub_categorias_info)
    except Exception as e:
        with open('file.html', 'w') as file:
            file.write(web.text)

In [152]:
web = requests_web('https://www.amazon.es/gp/site-directory?ref_=nav_em__fullstore_0_1_1_35')
soup = soup_file(web)
print('Buscando categorias en Amazon...')
all_categorias = get_all_categorias(soup)
print(f'Numero de categorias encontradas: {len(all_categorias)}')
print(f'Quitando categorias excluidas...')
categorias = excluir_categorias(all_categorias)
print(f'Numero de categorias actual: {len(categorias)}')
sub_categorias = verificar_sub_categoria_1(categorias)
sub_categorias_2 = verificar_sub_categoria_2(sub_categorias)
#print('Finalizado...')
#sub_categorias_2

200
Buscando categorias en Amazon...
Numero de categorias encontradas: 26
Quitando categorias excluidas...
Numero de categorias actual: 14

Titulo principal: Echo y Alexa
  └─ Cantidad Subcategorias: 17
  └─ Echo Flex
  └─ Echo Dot
  └─ Echo
  └─ Echo Plus
  └─ Echo Studio
  └─ Echo Show
  └─ Echo Show 5
  └─ Echo Show 8
  └─ Descubre todos los dispositivos con Alexa
  └─ Ring
  └─ Blink
  └─ Conoce a Alexa
  └─ App Alexa
  └─ Alexa integrada
  └─ Alexa y Hogar digital
  └─ Todas las Skills
  └─ Amazon Music Unlimited

Titulo principal: Amazon Fire TV
  └─ Cantidad Subcategorias: 6
  └─ Fire TV Stick con mando por voz Alexa
  └─ Fire TV Stick 4K
  └─ Accesorios
  └─ Apps y juegos para Fire TV Stick
  └─ Prime Video
  └─ Amazon Drive

Titulo principal: Tablets Fire
  └─ Cantidad Subcategorias: 8
  └─ Fire 7
  └─ Fire HD 8
  └─ Accesorios
  └─ Apps
  └─ Juegos
  └─ Música digital
  └─ eBooks Kindle
  └─ Gestionar contenido y dispositivos

Titulo principal: Ofertas y Chollos
  └─ Cantidad

TypeError: object of type 'NoneType' has no len()

In [ ]:
time.sleep(6)
categoria = get_sub_categoria('https://www.amazon.es/Accesorios-Informatica/b?ie=UTF8&node=2457643031&ref_=sd_allcat_pcacc')
categoria